In [1]:
import pandas as pd
from sqlalchemy import create_engine, insert, select
from sqlalchemy.orm import Mapped, mapped_column, sessionmaker, declarative_base, Session

#engine  = create_engine("sqlite:///:memory:", echo=True)
engine  = create_engine("sqlite:///models.db", echo=True, connect_args={"check_same_thread": False}) 
#connect_args={"check_same_thread": False} is for sqlite (https://fastapi.tiangolo.com/tutorial/sql-databases/)

from models import Results

#Session = sessionmaker(bind=engine)
Base = declarative_base()

In [10]:
#Base.metadata.create_all(engine)

2024-05-03 12:05:05,732 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-03 12:05:05,733 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("nab_results")
2024-05-03 12:05:05,733 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-05-03 12:05:05,734 INFO sqlalchemy.engine.Engine COMMIT


In [5]:
with Session(engine) as session:
    session.execute(
        insert(Results),
        values,
    )
    #session.add_all(values)
    session.commit()

2024-05-10 15:28:37,209 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-10 15:28:37,212 INFO sqlalchemy.engine.Engine INSERT INTO nab_results (id) VALUES (NULL)
2024-05-10 15:28:37,213 INFO sqlalchemy.engine.Engine [generated in 0.00128s] [(), (), (), (), (), (), (), ()  ... displaying 10 of 522 total bound parameter sets ...  (), ()]
2024-05-10 15:28:37,214 INFO sqlalchemy.engine.Engine ROLLBACK


IntegrityError: (sqlite3.IntegrityError) NOT NULL constraint failed: nab_results.detector
[SQL: INSERT INTO nab_results (id) VALUES (NULL)]
[parameters: [(), (), (), (), (), (), (), ()  ... displaying 10 of 522 total bound parameter sets ...  (), ()]]
(Background on this error at: https://sqlalche.me/e/20/gkpj)

In [3]:
session = Session(engine)

stmt = select(Results).where(Results.detector.in_(["bayesChangePt"]))

for user in session.scalars(stmt):
     print(user)
     print(user.filepath)

2024-05-10 15:27:11,421 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-10 15:27:11,425 INFO sqlalchemy.engine.Engine SELECT nab_results.id, nab_results.detector, nab_results.profile, nab_results.file_name, nab_results.dataset_group 
FROM nab_results 
WHERE nab_results.detector IN (?)
2024-05-10 15:27:11,426 INFO sqlalchemy.engine.Engine [generated in 0.00073s] ('bayesChangePt',)
<User(id=1, Detector=bayesChangePt, Profile=standard, Dataset_Group=artificialNoAnomaly, File Name=artificialNoAnomaly/art_daily_no_noise.csv)>
artificialNoAnomaly/artificialNoAnomaly/art_daily_no_noise.csv
<User(id=2, Detector=bayesChangePt, Profile=standard, Dataset_Group=artificialNoAnomaly, File Name=artificialNoAnomaly/art_daily_perfect_square_wave.csv)>
artificialNoAnomaly/artificialNoAnomaly/art_daily_perfect_square_wave.csv
<User(id=3, Detector=bayesChangePt, Profile=standard, Dataset_Group=artificialNoAnomaly, File Name=artificialNoAnomaly/art_daily_small_noise.csv)>
artificialNoAnomaly/artific

In [7]:
import os
import json
import pandas as pd
import config

ROOT_RESULTS=config.NAB_ASSETS / "results"
SAVE_DIRECTORY= config.NAB_ASSETS / "processed_results"

#GET FOLDERS FOR EACH MODEL
folders=next(os.walk(ROOT_RESULTS))[1]
profiles=[("Standard","_standard_scores.csv"),("Low FN","_reward_low_FN_rate_scores.csv"),("Low FP","_reward_low_FP_rate_scores.csv")]

df_all_results=pd.DataFrame()
for folder in folders: #for each model
    for name, file in profiles: #for each profile (name is profile name, file is the filename)
        filename=ROOT_RESULTS / folder / str(folder+file)
        df=pd.read_csv(filename).iloc[:-1,:] #remove last row that contained totals
        #add to a dataframe that stores all results
        df_all_results=pd.concat([df_all_results,df], axis=0)

df_all_results[['dataset_group', 'file_name']] = df_all_results['File'].str.split('/', expand=True)

values=df_all_results.to_dict('records')


In [8]:
values

[{'Detector': 'bayesChangePt',
  'Profile': 'standard',
  'File': 'artificialNoAnomaly/art_daily_no_noise.csv',
  'Threshold': 0.9846153846153848,
  'Score': -2.64,
  'TP': 0,
  'TN': 3404,
  'FP': 24,
  'FN': 0,
  'Total_Count': 3428,
  'dataset_group': 'artificialNoAnomaly',
  'file_name': 'art_daily_no_noise.csv'},
 {'Detector': 'bayesChangePt',
  'Profile': 'standard',
  'File': 'artificialNoAnomaly/art_daily_perfect_square_wave.csv',
  'Threshold': 0.9846153846153848,
  'Score': -2.64,
  'TP': 0,
  'TN': 3404,
  'FP': 24,
  'FN': 0,
  'Total_Count': 3428,
  'dataset_group': 'artificialNoAnomaly',
  'file_name': 'art_daily_perfect_square_wave.csv'},
 {'Detector': 'bayesChangePt',
  'Profile': 'standard',
  'File': 'artificialNoAnomaly/art_daily_small_noise.csv',
  'Threshold': 0.9846153846153848,
  'Score': -2.4200000000000004,
  'TP': 0,
  'TN': 3406,
  'FP': 22,
  'FN': 0,
  'Total_Count': 3428,
  'dataset_group': 'artificialNoAnomaly',
  'file_name': 'art_daily_small_noise.csv'}